In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import numpy as np

In [3]:
no_consec = pd.read_csv('../data/no_consec.csv')

C:\Users\jacob\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (17,40) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
no_consec = no_consec.rename(columns={'new_time_diff': 'hours_until_derate'})\
.drop(columns=['Unnamed: 0',
              'actionDescription',
               'index',
              'faultValue'])

In [5]:
no_consec['EventTimeStamp']= pd.to_datetime(no_consec['EventTimeStamp'])
no_consec['next_timestamp']= pd.to_datetime(no_consec['next_timestamp'])

In [6]:
no_consec = no_consec[(no_consec['EventTimeStamp'].dt.year >=2015) & (no_consec['EventTimeStamp'].dt.year <=2022)]

In [7]:
no_consec = no_consec[(no_consec['next_timestamp'].dt.year >=2015) & (no_consec['next_timestamp'].dt.year <=2022)]

In [8]:
pd.set_option('display.max_columns', 100)

In [9]:
no_consec = no_consec.reset_index(drop=True)

In [10]:
no_consec.head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,derate,full_derate,derate_cumulative,derate_cumu_diff,next_timestamp,time_diff_next,time_diff_next_h,updated_time,hours_until_derate
0,42692,2241075,2015-05-04 15:42:02,NaN,unknown,unknown,unknown,unknown,49,50353,0,False,2,1327,105383198,35.983796,-83.631944,2015-05-04 15:41:58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,255.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0.0,2015-05-04 15:38:35,0 days 00:03:27,0.057500,NaN,NaN
1,45667,2296851,2015-05-07 06:52:14,Condition Exists Cruise Control Enable Switch,unknown,unknown,unknown,unknown,49,596,31,True,126,1327,105383198,40.197500,-74.661435,2015-05-07 06:52:50,0.0,14.790,False,64.6226,518554.7,185.0,14.0,25.52,165.8750,648.375,10338.7,NaN,78990.877785,0.766101,NaN,True,102.2,255.0,False,NaN,0.00000,3276.75,NaN,1.74,0,0,0,0.0,2015-05-04 15:42:02,2 days 15:10:12,63.170000,NaN,NaN
2,45945,2302281,2015-05-07 10:25:59,Condition Exists Cruise Control Enable Switch,unknown,unknown,unknown,unknown,49,596,31,False,126,1327,105383198,40.198148,-74.660879,2015-05-07 10:25:55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0.0,2015-05-07 06:52:14,0 days 03:33:45,3.562500,NaN,NaN
3,58809,2507532,2015-05-19 12:02:55,Abnormal Update Rate Aftertreatment 1 Intake NOx,04993120*00001782*082113134117*07700053*I0*BBZ*,79419774,6X1u10D1500000000,CMMNS,0,3216,9,True,1,1327,105383198,35.491250,-86.458842,2015-05-19 12:03:32,11.6,14.355,False,64.6226,520309.6,185.0,0.0,31.90,203.8438,1083.125,10374.8,NaN,79274.466482,0.000000,NaN,True,109.4,17407.0,False,NaN,27.56364,3276.75,NaN,2.90,0,0,0,0.0,2015-05-07 10:25:59,12 days 01:36:56,289.615556,NaN,NaN
4,65879,2610228,2015-05-26 08:11:45,Abnormal Update Rate Aftertreatment 1 Intake NOx,04993120*00001782*082113134117*07700053*I0*BBZ*,79419774,6X1u10D1500000000,CMMNS,0,3216,9,True,1,1327,105383198,35.490787,-86.433842,2015-05-21 14:40:18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,17407.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0.0,2015-05-19 12:02:55,6 days 20:08:50,164.147222,NaN,NaN


In [11]:
derates = no_consec[no_consec['hours_until_derate'].notna()]

# Filling Distances

In [14]:
from geopy.distance import geodesic
from tqdm import tqdm
import time

In [18]:
derates['dist_from_last'] = 0
for i in derates.index[0:9]:
    first = (derates.iloc[i].Latitude, derates.iloc[i].Longitude)
    sec = (derates.iloc[i-1].Latitude, derates.iloc[i-1].Longitude)
    derates.loc[derates.index == i,'dist_from_last'] = geodesic(first, sec).miles

<ipython-input-18-79c8b8175328>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  derates['dist_from_last'] = 0
C:\Users\jacob\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\jacob\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

In [20]:
derates.head(10)

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,derate,full_derate,derate_cumulative,derate_cumu_diff,next_timestamp,time_diff_next,time_diff_next_h,updated_time,hours_until_derate,dist_from_last
47,5713,1070644,2015-02-25 13:53:08,NaN,unknown,unknown,unknown,unknown,0,4344,5,True,1,1329,105400037,39.399583,-82.974768,2015-02-25 13:56:31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,22527.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0.0,2015-04-15 08:58:11,-49 days +04:54:57,0.000000,0.0,0.0,57.006919
48,5714,1070646,2015-02-25 13:53:08,NaN,unknown,unknown,unknown,unknown,0,5246,0,True,1,1329,105400037,39.399583,-82.974768,2015-02-25 13:56:31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,22527.0,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,1.0,2015-02-25 13:53:08,0 days 00:00:00,0.000000,0.0,0.0,0.710136
49,5715,1070647,2015-02-25 13:53:08,Condition Exists Engine Protection Torque Derate,unknown,unknown,unknown,unknown,0,1569,31,True,1,1329,105400037,39.399583,-82.974768,2015-02-25 13:56:31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,22527.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,1,0.0,2015-02-25 13:53:08,0 days 00:00:00,0.000000,0.0,0.0,30.066305
50,5716,1070648,2015-02-25 13:53:08,Low (Severity Medium) Catalyst Tank Temperature,unknown,unknown,unknown,unknown,0,3031,18,True,1,1329,105400037,39.399583,-82.974768,2015-02-25 13:56:31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,22527.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0.0,2015-02-25 13:53:08,0 days 00:00:00,0.000000,0.0,0.0,1.230863
51,5811,1071908,2015-02-25 14:47:20,Condition Exists Engine Protection Torque Derate,unknown,unknown,unknown,unknown,0,1569,31,False,1,1329,105400037,39.399629,-82.974814,2015-02-25 14:47:16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17407.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,1,0.0,2015-02-25 13:53:08,0 days 00:54:12,0.903333,0.0,0.0,70.243403
52,5899,1073059,2015-02-25 15:38:07,High (Severity Low) Aftertreatment 1 Particula...,unknown,unknown,unknown,unknown,0,3242,15,True,1,1329,105400037,39.399583,-82.974861,2015-02-25 15:38:44,0.0,14.2825,False,59.03026,483744.4,204.8,34.0,22.62,220.7188,898.375,9337.00,70.0,72585.630126,5.111745,NaN,True,100.4,17407.0,True,NaN,0.0,3276.75,NaN,13.63,0,0,1,0.0,2015-02-25 14:47:20,0 days 00:50:47,0.846389,0.0,0.0,0.026548
53,6183,1076232,2015-02-25 19:36:42,High (Severity Low) Aftertreatment 1 Particula...,unknown,unknown,unknown,unknown,0,3242,15,False,1,1329,105400037,39.399537,-82.974768,2015-02-25 19:36:38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0.0,2015-02-25 15:38:07,0 days 03:58:35,3.976389,0.0,0.0,360.432510
54,6184,1076233,2015-02-25 19:36:42,Low (Severity Medium) Catalyst Tank Temperature,unknown,unknown,unknown,unknown,0,3031,18,False,1,1329,105400037,39.399537,-82.974768,2015-02-25 19:36:38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0.0,2015-02-25 19:36:42,0 days 00:00:00,0.000000,0.0,0.0,1.809683
55,6185,1076234,2015-02-25 19:36:42,NaN,unknown,unknown,unknown,unknown,0,4344,5,False,1,1329,105400037,39.399537,-82.974768,2015-02-25 19:36:38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0.0,2015-02-25 19:36:42,0 days 00:00:00,0.000000,0.0,0.0,45.882291
56,21204,1869361,2015-04-14 01:31:28,Incorrect Data Front Operator Wiper Switch,unknown

In [23]:
geodesic((39.399537,-82.974768),(39.399583,-82.974861)).miles

0.005903163488252717

# Simple Linear Regression
- no imputation

In [72]:
from sklearn.linear_model import LinearRegression, RANSACRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer

In [25]:
features_from_eda = ['DistanceLtd',
                     'EngineCoolantTemperature',
                     'FuelLtd', 
                    'spec_spn']

In [26]:
error_messages = derates[(derates['updated_time'] !=0) & (derates['full_derate']==0)]

In [27]:
error_messages.shape

(221417, 52)

In [56]:
error_messages = error_messages[error_messages[features_from_eda[0]].notna()]

In [57]:
#error_messages = error_messages[error_messages[features_from_eda[1]].notna()]

In [58]:
#error_messages = error_messages[error_messages[features_from_eda[2]].notna()]

In [59]:
#error_messages

In [60]:
important_spns =[3362, 1761, 1569, 111, 1068]
unimportant = error_messages[~error_messages.isin(important_spns)]['spn'].unique().tolist()

In [61]:
cond = [error_messages['spn'].isin(important_spns), (error_messages['spn'].isin(unimportant))]
choices = [error_messages['spn'], 0]

error_messages['spec_spn'] = np.select(cond, choices)

In [62]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer, KNNImputer

In [63]:
X = error_messages[features_from_eda]
y = error_messages['hours_until_derate']

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)

In [64]:
ohe = OneHotEncoder(sparse = False, drop = 'first')
ohe.fit(X_train[['spec_spn']])

OneHotEncoder(drop='first', sparse=False)

In [65]:
ohe.transform(X_train[['spec_spn']])

array([[1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]])

In [66]:
ct = ColumnTransformer(transformers = [
    ('ohe', OneHotEncoder(sparse = False, drop = 'first'), ['spec_spn'])
],
                      remainder = 'passthrough')

In [67]:
steps = [('ct', ct),
         ('imputer', KNNImputer()),
    ('scaler', StandardScaler()),
         ('pf', PolynomialFeatures(interaction_only = True, include_bias = False)),
          ('linreg', LinearRegression())]

In [68]:
pipe = Pipeline(steps)

pipe.fit(X_train, y_train)

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(drop='first',
                                                                sparse=False),
                                                  ['spec_spn'])])),
                ('imputer', KNNImputer()), ('scaler', StandardScaler()),
                ('pf',
                 PolynomialFeatures(include_bias=False, interaction_only=True)),
                ('linreg', LinearRegression())])

In [69]:
print(f'MSE: {mean_squared_error(y_test, pipe.predict(X_test))}')
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

MSE: 35748559.56913775
R2: 0.21856718396903463


In [70]:
np.sqrt(35748559.56913775) /24

249.1254131440578

- off by 250 days on average, need it to be much better.

# Trying Hist Gradient Boosting

In [229]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor, GradientBoostingRegressor

In [286]:
from sklearn.feature_selection import SelectKBest, f_regression

In [292]:
corrmatrix = error_messages.corr()

corrmatrix.style.background_gradient('coolwarm')

C:\Users\jacob\anaconda3\lib\site-packages\pandas\io\formats\style.py:1264: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(s.to_numpy()) if vmin is None else vmin
C:\Users\jacob\anaconda3\lib\site-packages\pandas\io\formats\style.py:1265: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(s.to_numpy()) if vmax is None else vmax


,RecordID,ESS_Id,ecuSource,spn,fmi,active,activeTransitionCount,MCTNumber,Latitude,Longitude,AcceleratorPedal,BarometricPressure,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IntakeManifoldTemperature,LampStatus,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,derate,full_derate,derate_cumulative,derate_cumu_diff,time_diff_next_h,updated_time,hours_until_derate,dist_from_last,spec_spn
RecordID,1.000000,0.833301,-0.010991,-0.011544,-0.125828,nan,0.222627,0.140757,-0.012393,-0.012566,-0.094352,-0.008234,-0.115392,0.086793,-0.054420,-0.075898,-0.018830,-0.010008,-0.066022,0.082668,0.024125,-0.010359,-0.008920,0.034530,-0.022002,0.254196,nan,-0.008348,-0.095938,0.484615,-0.049118,0.031522,nan,0.475622,nan,0.081015,0.081015,-0.237928,nan,0.017674
ESS_Id,0.833301,1.000000,0.040514,0.002090,-0.147093,nan,0.166103,0.202019,-0.012446,-0.031513,-0.082564,-0.005347,-0.116589,0.043629,-0.040996,-0.081073,-0.017744,-0.006226,-0.060538,0.059375,0.005787,-0.006523,-0.005416,-0.012400,-0.003744,0.273254,nan,-0.005271,-0.078161,0.345214,-0.044814,0.051622,nan,0.451628,nan,0.102700,0.102700,-0.196812,nan,0.044253
ecuSource,-0.010991,0.040514,1.000000,0.062272,-0.581494,nan,0.551544,-0.081686,-0.001551,0.032291,-0.118798,-0.004568,0.097154,-0.043213,-0.059713,-0.155592,-0.020198,-0.003173,-0.079180,-0.046105,-0.031825,-0.002630,-0.002002,-0.207989,0.210958,-0.167829,nan,0.000369,0.008448,-0.331391,-0.075175,-0.064959,nan,0.142419,nan,-0.006536,-0.006536,0.106882,nan,-0.191589
spn,-0.011544,0.002090,0.062272,1.000000,-0.042074,nan,-0.035753,0.002863,-0.007984,-0.002786,-0.019553,0.001388,-0.014338,0.027646,0.001054,-0.014345,-0.002725,0.001369,-0.008510,0.019824,0.005077,0.001175,0.000584,0.054046,0.011828,0.043128,nan,0.000534,-0.002329,-0.024133,-0.006555,0.007108,nan,-0.053139,nan,0.036960,0.036960,-0.023605,nan,-0.003503
fmi,-0.125828,-0.147093,-0.581494,-0.042074,1.000000,nan,-0.411734,0.052698,-0.011862,-0.017614,0.172353,-0.004383,0.094305,0.074344,0.033537,0.165672,0.010582,-0.003193,0.073466,0.080644,0.043934,-0.001000,-0.000593,-0.051626,-0.200811,-0.114389,nan,-0.001786,0.018284,0.249005,0.077455,0.295947,nan,-0.212578,nan,-0.019924,-0.019924,-0.042346,nan,0.342373
active,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
activeTransitionCount,0.222627,0.166103,0.551544,-0.035753,-0.411734,nan,1.000000,0.036130,0.002274,0.030891,-0.073679,0.004341,-0.050651,-0.461828,-0.028502,-0.125426,0.013207,0.004411,-0.053852,-0.342118,0.086027,0.005909,0.004084,-0.275154,0.187903,-0.134025,nan,0.004978,0.010813,0.029884,-0.061233,-0.099740,nan,0.512062,nan,-0.051449,-0.051449,0.159719,nan,-0.172047
MCTNumber,0.140757,0.202019,-0.081686,0.002863,0.052698,nan,0.036130,1.000000,-0.000430,0.016802,-0.039052,-0.002040,-0.240469,-0.130069,-0.018647,-0.041109,-0.013593,-0.002283,-0.031567,-0.084476,0.016044,-0.002414,-0.001932,-0.003001,0.015809,0.331338,nan,-0.001966,-0.066137,0.107636,-0.019272,-0.010531,nan,0.158274,nan,0.003006,0.003006,-0.017760,nan,-0.001469
Latitude,-0.012393,-0.012446,-0.001551,-0.007984,-0.011862,nan,0.002274,-0.000430,1.000000,0.348881,0.001641,-0.005940,-0.006821,0.000172,-0.014106,0.009768,-0.004254,-0.006330,0.002329,-0.011577,0.012620,-0.007116,-0.005352,-0.001327,-0.138294,-0.019400,nan,-0.005391,-0.013210,-0.019359,0.003495,0.001327,nan,-0.004245,nan,-0.001804,-0.001804,-0.000443,nan,0.007344
Longitude,-0.012566,-0.031513,0.032291,-0.002786,-0.017614,nan,0.030891,0.016802,0.348881,1.000000,-0.003163,-0.002806,0.048074,0.012577,-0.010975,-0.009166,-0.004887,-0.002970,-0.002808,-0.002489,-0.026055,-0.003064,-0.002524,-0.046174,-0.044882,-0.023045,nan,-0.002402,-0.007336,0.038005,-0.000718,-0.004262,nan,-0.034823,nan,-0.009099,-0.009099,-0.022667,nan,-0.012815


In [480]:
features_from_eda = ['DistanceLtd',
                     #'EngineCoolantTemperature', 
                     'EngineTimeLtd',
                     'FuelTemperature',
                     'activeTransitionCount',
                     'Throttle',
                     'CruiseControlSetSpeed',
                     #'EngineLoad',
                    'spn']

In [471]:
X = error_messages[features_from_eda]
y = error_messages['hours_until_derate']

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)

In [481]:
steps2 = [('scaler', MinMaxScaler()),
          ('hist', HistGradientBoostingRegressor(max_iter = 1000, l2_regularization=10))]

In [482]:
pipe_hist = Pipeline(steps2)

In [483]:
pipe_hist.fit(X_train, y_train)

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('hist',
                 HistGradientBoostingRegressor(l2_regularization=10,
                                               max_iter=1000))])

In [484]:
y_pred = pipe_hist.predict(X_test)

In [485]:
difference_array = np.subtract(np.array(y_test), y_pred)
squared_array = np.square(difference_array)
mse = squared_array.mean()

In [486]:
print(f'MSE: {mse}')

MSE: 8207977.094858219


In [487]:
np.sqrt(mse)/24

119.37319730211156

In [488]:
r2_score(y_test, y_pred)

0.8205806686351076